In [1]:
import json
import pandas as pd
import pprint
import numpy as np
import labelme
import base64
from tqdm import tqdm

pp = pprint.PrettyPrinter(indent=4)
file_path = "/data2/itri/rosbag_api_sync/05051602_16202024975019_16202025296023/2_2/16202024975019.json"


In [2]:
with open(file_path, "r") as f:
	data = json.load(f)

# print(data.keys())

output_dict = dict(
	version="4.5.13",
  flags={},
)

# print(json.dumps(data, indent=2))


In [3]:
def get_bbox_2pt(four_pt_list):
	# u: left/right, v: up/down
	def u_coord(x):
		return x/0.175 + 1142/2
	def v_coord(y):
		return -y/0.175 + 1142

	x_min = int(u_coord(min([x[0] for x in four_pt_list])))
	y_min = int(v_coord(min([x[1] for x in four_pt_list])))
	x_max = int(u_coord(max([x[0] for x in four_pt_list])))
	y_max = int(v_coord(max([x[1] for x in four_pt_list])))

	return [[x_min, y_min], [x_max, y_max]]


def get_bbox_xy(data, lidar_type):
	all_bbox = []
	for i, b in enumerate(data['bbox']): # for each bbox
		bbox = b[:, [0, 1]]
		if lidar_type == "baraja":
			x = -bbox[:, [1]]-0.03
			y = bbox[:, [0]]+0.6
			bbox = np.hstack((x, y))
		if lidar_type == "livox":
			x = -bbox[:, [1]]+0.18
			y = bbox[:, [0]]+0.6
			bbox = np.hstack((x, y))
		if lidar_type == "all":
			x = -bbox[:, [1]]-0.1
			y = bbox[:, [0]]+0.6
			bbox = np.hstack((x, y))

		four_pt_list = set(tuple(item) for item in bbox)
		single_anno = dict(
			# label = "car",
			label = int(data['cls'][i]),
			points = get_bbox_2pt(four_pt_list),
			group_id = None,
			shape_type = "rectangle",
			flags = {}
		)
		all_bbox.append(single_anno)
		# print(single_anno)
	# print("all_box: ", all_bbox)
	return all_bbox

In [4]:
a = np.random.randint(0, 10, (3, 2))
print(a)

print(a[:, [1, 0]])
print(a[:, [0, 1]])

[[4 2]
 [5 2]
 [4 2]]
[[2 4]
 [2 5]
 [2 4]]
[[4 2]
 [5 2]
 [4 2]]


### combine two sensors' bbox

In [8]:
import os
baraja_bbox_path = "/data2/itri/RAV4/2021-11-24-11-38-59/processing/0_200/baraja_bbox_npy"
livox_bbox_path = "/data2/itri/RAV4/2021-11-24-11-38-59/processing/0_200/livox_bbox_npy"
radar_img_path = "/data2/itri/RAV4/2021-11-24-11-38-59/processing/0_200/radar_lidar_overlay"

baraja_file = os.listdir(baraja_bbox_path)
baraja_file.sort()

for bf in tqdm(baraja_file[:]):
	all_bbox = []

	data = np.load(os.path.join(
		baraja_bbox_path,
		bf
	))

	print(data)

	bbox_1 = get_bbox_xy(data, "baraja")

	data = np.load(os.path.join(
		livox_bbox_path,
		bf
	))
	bbox_2 = get_bbox_xy(data, "livox")

	all_bbox = bbox_1 + bbox_2
	# print(all_bbox)

	im_data = labelme.LabelFile.load_image_file(
		os.path.join(
			radar_img_path,
			bf[:-4]+".png"
		)
	)
	image_data = base64.b64encode(im_data).decode('utf-8')

	out_json = {
		"version": "4.5.13",
    "flags": {},
		"shapes": all_bbox,
		"imagePath": bf[:-4]+".png",
		"imageData": image_data,
		"imageHeight": 1142,
		"imageWidth": 1142
	}

	# print(all_bbox)

	break

	# with open("/data2/itri/RAV4/2021-11-24-11-38-59/processing/0_200/radar_lidar_overlay_anno/{}.json".format(bf[:-4]), "w") as f:
		# json.dump(out_json, f, indent = 2)


	# print(json.dumps(out_json, indent=2))

  0%|          | 0/2323 [00:00<?, ?it/s]

[[[22.14640039 11.46911649 -1.56747645]
  [22.14539101  9.88366771 -1.56747645]
  [22.14539101  9.88366771 -1.56747645]
  [18.42903906  9.88603374 -1.56747645]
  [18.42903906  9.88603374 -1.56747645]
  [18.43004844 11.47148251 -1.56747645]
  [18.43004844 11.47148251 -1.56747645]
  [22.14640039 11.46911649 -1.56747645]]]


  0%|          | 0/2323 [00:00<?, ?it/s]


### use single repo to gen labelme bbox

In [11]:
import os
# baraja_bbox_path = "/data2/itri/RAV4/2021-11-24-11-38-59/processing/0_200/baraja_bbox_npy"
# livox_bbox_path = "/data2/itri/RAV4/2021-11-24-11-38-59/processing/0_200/livox_bbox_npy"
radar_img_path = "/data2/itri/RAV4/2021-11-24-11-38-59/radar_timelapsed/0_200/no_comp/radar_lidar_overlay"
# bbox_npy_path = "/data2/itri/RAV4/2021-11-24-11-38-59/radar_timelapsed/0_200/no_comp/baraja_livox_bbox_npy"
bbox_npy_path = "/data2/itri/RAV4/2021-11-24-11-38-59/radar_timelapsed/0_200/no_comp/for_bimo_centernet/baraja_livox_bbox_npy"
out_json_path = '/data2/itri/RAV4/2021-11-24-11-38-59/radar_timelapsed/0_200/no_comp/for_bimo_centernet/labelme_json'

lidar = os.listdir(bbox_npy_path)
lidar.sort()

for lf in tqdm(lidar[:]):
	data = np.load(os.path.join(
		bbox_npy_path,
		lf
	))

	# print(lf)

	bbox_1 = get_bbox_xy(data, "all")

	# if lf == '16377251434601.npz':
	# 	print(len(bbox_1))

	all_bbox = bbox_1
	# im_data = labelme.LabelFile.load_image_file(
	# 	os.path.join(
	# 		radar_img_path,
	# 		lf[:-4]+".png"
	# 	)
	# )

	# image_data = base64.b64encode(im_data).decode('utf-8')

	out_json = {
		"version": "4.5.13",
    "flags": {},
		"shapes": all_bbox,
		"imagePath": lf[:-4]+".png",
		# "imageData": image_data,
		"imageHeight": 1142,
		"imageWidth": 1142
	}

	with open("{}/{}.json".format(out_json_path, lf[:-4]), "w") as f:
		json.dump(out_json, f, indent = 2)


	# print(json.dumps(out_json, indent=2))

100%|██████████| 2321/2321 [00:06<00:00, 374.50it/s]


In [60]:
import labelme
import base64

data = labelme.LabelFile.load_image_file(
	os.path.join(
		radar_img_path,
		bf[:-4]+".png"
	)
)
image_data = base64.b64encode(data).decode('utf-8')
# print(image_data)
